In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("meowmeowmeowmeowmeow/gtsrb-german-traffic-sign")

print("Path to dataset files:", path)

In [ ]:
import os

data_path = "/kaggle/input/gtsrb-german-traffic-sign"
print(os.listdir(data_path))  # Liệt kê các file/folder trong dataset


In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image , ImageOps
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,Dense, MaxPool2D , Dropout, Flatten
from tensorflow.keras.models import Model
from sklearn.metrics import accuracy_score
 


In [ ]:
import os
import numpy as np
from PIL import Image, ImageOps  # Import thư viện PIL

# Đường dẫn chính xác đến thư mục Train
train_dir = "/kaggle/input/gtsrb-german-traffic-sign/Train"

data = []
labels = []
classes = 43

# Duyệt qua từng class (0-42)
for i in range(classes):
    path = os.path.join(train_dir, str(i))   
    if not os.path.exists(path):  # Kiểm tra nếu thư mục không tồn tại
        print(f"⚠️ Cảnh báo: Thư mục {path} không tồn tại!")
        continue
    
    images = os.listdir(path)  # Danh sách ảnh trong thư mục
    for a in images:
        try:
            img_path = os.path.join(path, a)  # Đường dẫn ảnh
            image = Image.open(img_path).convert("RGB")  # Chuyển ảnh về RGB
            image = image.resize((32,32))  # Resize về (32,32)

            # Chuẩn hóa ảnh về [0,1]
            image_np = np.array(image) / 255.0  
            data.append(image_np)
            labels.append(i)

          
            ## **Thêm ảnh xoay 15 độ**
            rotated_image = image.rotate(15)  #
            rotated_np = np.array(rotated_image) / 255.0
            data.append(rotated_np)
            labels.append(i)
        ## **Thêm ảnh xoay -15 độ**
            rotated_image_neg = image.rotate(-15)  
            rotated_np_neg = np.array(rotated_image_neg) / 255.0
            data.append(rotated_np_neg)
            labels.append(i)
           
        except Exception as e:
            print(f"❌ Lỗi khi xử lý ảnh {img_path}: {e}")
 

print("✅ Load dữ liệu thành công!")



In [ ]:
# Chuyển list thành numpy array
data = np.array(data,dtype="float32")
labels = np.array(labels)
data.shape, labels.shape



In [ ]:

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
X_train.shape, X_test.shape 


In [ ]:
y_train_one_hot = to_categorical(y_train, 43)
y_test_one_hot = to_categorical(y_test, 43)
print("Dạng one-hot của nhãn train:", y_train_one_hot.shape)
print("Dạng one-hot của nhãn test:", y_test_one_hot.shape)
 

In [ ]:
#Building the model
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=64, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.3))
model.add(Flatten())                                             
model.add(Dense(250, activation='relu'))                               
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))


In [ ]:
# Vẽ biểu đồ Accuracy
plt.figure(figsize=(6,4))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.legend()
plt.show()
# Vẽ biểu đồ Loss
plt.figure(figsize=(6,4))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Model Loss')
plt.legend()
plt.show()
 

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.metrics import accuracy_score

# Đọc file CSV chứa thông tin test
y_test = pd.read_csv("/kaggle/input/gtsrb-german-traffic-sign/Test.csv")
labels = y_test["ClassId"].values  
imgs = y_test["Path"].values       

 
base_dir = "/kaggle/input/gtsrb-german-traffic-sign/"
data = []

# Xử lý từng ảnh
for img in imgs:
    full_path = base_dir + img   
    try:
        image = Image.open(full_path).convert("RGB")   
        image = image.resize((32, 32))                
        image = np.array(image) / 255.0             
        data.append(image)                           
    except FileNotFoundError:
        print(f"Không tìm thấy file: {full_path}")
        continue

# Chuyển data thành mảng NumPy để dự đoán
X_test = np.array(data)
pred = np.argmax(model.predict(X_test), axis=1)  # Dự đoán với model (giả sử model đã được định nghĩa)
print("Độ chính xác:", accuracy_score(labels, pred))  # In độ chính xác